In [1]:
import pandas as pd

In [2]:
bbtea = pd.read_json('ns-names.json')
#bbtea['address'] = bbtea['address'].apply(lambda x: x.replace('[','').replace(']','')) 
bbtea_col = pd.DataFrame(bbtea.address.tolist(), index= bbtea.index)
bbtea_col = bbtea_col.replace(to_replace= '\n', value= '', regex=True)

In [3]:
bbtea_col = bbtea_col.rename(columns={1: 'address'})
bbtea_col = bbtea_col['address'] #keeping only address column

In [4]:
bbtea = pd.concat([bbtea, pd.DataFrame(bbtea_col)], axis=1) #df with names + addresses

In [5]:
#adding lat, long in order to be able to map with plotly
import requests

def Coords():
    coord_list = []
    for index, row in bbtea.iterrows():
        address = row['address']
        api = 'http://api-adresse.data.gouv.fr/search/'
        response = requests.get(api, params={'q': address, 'limit': 1})
        response2 = response.json()
        coord = response2['features'][0]['geometry']['coordinates'] #getting coordinates, wrong order though!
        coord.reverse()
        coord_list.append(coord)

    return coord_list

coord_list = Coords()
bbtea['Coordinates'] = coord_list
bbtea['Coordinates'] = bbtea['Coordinates'].astype(str).apply(lambda x: x.replace('[','').replace(']','')) 
bbtea['Lat'], bbtea['Long'] = bbtea['Coordinates'].str.split(',', 1).str
bbtea['Lat'] = bbtea['Lat'].astype(float)
bbtea['Long'] = bbtea['Long'].astype(float)
bbtea

,name,address,address,Coordinates,Lat,Long
0,糖轩 T'Xuan,"[\n , \n 56 Rue la Fayet...","56 Rue la Fayette, 75009 Paris ...","48.875299, 2.342124",48.875299,2.342124
1,machi machi Paris,"[\n , \n 5 Rue Chabanais...","5 Rue Chabanais, 75002 Paris ...","48.867285, 2.336732",48.867285,2.336732
2,甜觅茶室 Long & Tee,"[\n , \n 39 Rue la Fayet...","39 Rue la Fayette, 75009 Paris ...","48.874694, 2.339004",48.874694,2.339004
3,旺角茶餐厅 Mongkok Resto,"[\n , \n 23 Rue Juge, 75...","23 Rue Juge, 75015 Paris","48.849894, 2.29319",48.849894,2.293190
4,初代抹茶甜品店 Matcha,"[\n , \n 7 Rue Volta, 75...","7 Rue Volta, 75003 Paris","48.864887, 2.356882",48.864887,2.356882
...,...,...,...,...,...,...
70,熊猫煎饼 Panda Crêpe,"[\n , \n 12 Rue Volta, 7...","12 Rue Volta, 75003 Paris","48.864841, 2.356983",48.864841,2.356983
71,日出茶太 Chatime (13区),"[\n , \n 124 Avenue de C...","124 Avenue de Choisy, 75013 Paris ...","48.826668, 2.359739",48.826668,2.359739
72,台北厨房 Taipei Gourmet,"[\n , \n 5 Boulevard Sai...","5 Boulevard Saint Marcel, 75013 Pa...","48.839391, 2.360686",48.839391,2.360686
73,糖楼 Sweet,"[\n , \n Centre Commerci...",Centre Commercial OSLO 44 Avenue d...,"48.823457, 2.365802",48.823457,2.365802


In [6]:
bbtea.isnull().sum() #all good, no null values, all coodinates were found

name           0
address        0
address        0
Coordinates    0
Lat            0
Long           0
dtype: int64

In [9]:
#mapping locations in paris
import plotly.express as px

fig = px.scatter_mapbox(bbtea, lat="Lat", lon="Long", hover_name="name",
                    zoom=11, height=300)
fig.update_layout(mapbox_style="carto-positron")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()